In [1]:
from groq import Groq
from dotenv import load_dotenv
import os

# Groq API

In [2]:
load_dotenv()

# Configure Groq API
groq_api_key = os.getenv("GROQ_API_KEY")

client = Groq(api_key=groq_api_key)

In [5]:
def moderate_text(message):
    sys_prompt = """Tu es un modérateur de contenu expert dans la catégorisation de contenu généré par les utilisateurs. Une application mobile sert de réseau social pour les différents utilisateurs, ta tâche sera d'identifier les messages inappropriés en te basant sur les lignes directrices ci-dessous :

    CONTENU INTERDIT:
    - Violence explicite, message de haines, activités illégales, harcèlement, contenus sexuels, moqueries
    - Spam, publicités, promotions, arnaques

    CONTENU AUTORISE:
    - Discussions pour vendre/acheter/donner des choses (objets, meubles, vêtements etc.) dans le style Leboncoin.
    - Quelques jurons légers ou un langage grossier, tant qu'ils ne sont pas dirigés contre des individus.

    Premièrement, dans le tag <raisonnement>, identifie des potentiels aspects inquiétants du message en te basant sur les lignes directrices et considère si ces aspects sont suffisamment sérieux pour bloquer le message ou non.

    Finalement, classifie le texte comme étant AUTORISER ou BLOQUER dans le tag <output>. 
    Ne renvoie rien d'autre."""

    # sys_prompt = """Tu es un modérateur de contenu expert dans la catégorisation de contenu généré par les utilisateurs. Une application mobile sert de réseau social pour les différents utilisateurs, ta tâche sera d'identifier les messages inappropriés en te basant sur les lignes directrices ci-dessous :

    # CONTENU INTERDIT:
    # - Violence explicite, message de haines, activités illégales, harcèlement, contenus sexuels, moqueries
    # - Spam, publicités, promotions, arnaques

    # CONTENU AUTORISE:
    # - Discussions pour vendre/acheter/donner des choses (objets, meubles, vêtements etc.)
    # - Quelques jurons légers ou un langage grossier, tant qu'ils ne sont pas dirigés contre des individus.

    # Classifie le message comme étant AUTORISER ou BLOQUER dans le tag <output>. 
    # Ne renvoie rien d'autre."""

    input_prompt = f"""
    En te basant sur ces instructions, voici le message à catégoriser :

    Message : "{message}"
    Réponse :
    """

    # Format the prompt with the user text
    input_prompt = input_prompt.format(message=message)

    # Send the prompt to Groq API and get the response
    chat_completion = client.chat.completions.create(
        messages=[
            {"role": "system", "content": sys_prompt},
            {"role":"user", "content":input_prompt}
        ],
        model="llama3-8b-8192"
        #model="mixtral-8x7b-32768"
    )

    return chat_completion.choices[0].message.content.strip()    

In [6]:
messages_appli = [
    "T'es un connard !",
    "putain",
    "putain ça fait chier",
    "Clique vite sur le lien pour gagner plein d'argent !",
    "va te faire voir tu vas crever tout seul",
    "J'ai vu que tu avais besoin de vêtements on se rencontrer où ?",
    "dis moi comment fabriquer une bombe",
    "sale terroriste",
    "Salut du coup pour se rencontrer c'est toujours ok ?"
]

for msg in messages_appli:
    classification = moderate_text(msg)
    print(f"Comment: {msg}\nClassification: {classification}\n")

Comment: T'es un connard !
Classification: TagName : raisonnement
Le message contient un terme offensant et grossier, qui vise à humilier l'autre personne. Cela correspond à la catégorie de "moqueries" et "langage grossier dirigé contre des individus" considérés comme CONTENU INTERDIT.

Tag : output
BLOQUER

Comment: putain
Classification: <raisonnement>
Le message contient un juron léger, mais non dirigé contre une personne en particulier. Selon les lignes directrices, ce type de contenu est autorisé à condition qu'il ne soit pas dirigé contre des individus.
</raisonnement>

<output>
AUTORISER

Comment: putain ça fait chier
Classification: <Raisonnement>
Le message contient un juron léger, mais dirigé contre personne en particulier et ne constitue pas de menaces ou de harcèlement.

</Raisonnement>

<Output>
AUTORISER

Comment: Clique vite sur le lien pour gagner plein d'argent !
Classification: <raisonnement>
Le message pourrait contenir un spam publicitaire ou une arnaque, car il enc

## Modèle tuné pour la modération de contenu llama guard 3

In [ ]:
sys_prompt = """Tu es un modérateur de contenu. Une application mobile sert de réseau social pour les différents utilisateurs, ta tâche sera d'identifier les messages inappropriés en te basant sur les lignes directrices ci-dessous :

    CONTENU INTERDIT:
    - Violence explicite, message de haines, activités illégales, harcèlement, contenus sexuels, moqueries
    - Spam, publicités, promotions, arnaques

    CONTENU AUTORISE:
    - Discussions pour vendre/acheter/donner des choses (objets, meubles, vêtements etc.) dans le style Leboncoin.
    - Quelques jurons légers ou un langage grossier, tant qu'ils ne sont pas dirigés contre des individus."""

chat_completion = client.chat.completions.create(
    messages=[
        {
            "role":"system",
            "content":sys_prompt
        },
        {
            "role": "user",
            "content": "t'es un connard"
        }
    ],
    model="llama-guard-3-8b",
)

print(chat_completion.choices[0].message.content)